<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/Query_Decomposition/Query_decompose_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install -q groq
!pip install -U accelerate bitsandbytes datasets evaluate
!pip install -U peft transformers trl
import openai
import time
from tqdm import tqdm
!pip install openai==0.28

In [2]:
# For Google Colab settings
from google.colab import userdata, drive

# This will prompt for authorization
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json
Multihop_path = "/content/drive/MyDrive/NER/MultiHopRAG.json"

# Load the HotpotQA JSON file
with open(Multihop_path, 'r') as file:
    hotpotqa_data = json.load(file)

In [4]:
openai.api_key = userdata.get('OPENAI_API_KEY')

In [5]:
sampler = []
max_num=0
# Transform the data and save it to sampler
max_num=0
for item in hotpotqa_data:
    question = item['query']

    # Add to sampler
    sampler.append(question)
    max_num += 1
    if max_num == 301:
        break

sampler_length = len(sampler)

In [6]:
system_message = """
    You are an experienced Query decomposer.
    You need to break down multiple composite queries and change them to a single question.
    The reason for dividing a query is to divide it into a single question so that when searching for a query, there are no questions that are marginalized by complex questions.
    For example, if you have a query, "What is the CEO name of Apple and Google and Samsung?" you have to divide it into "What is the CEO name of Apple?" "What is the CEO name of Google?" and "What is the CEO name of Samsung?"


    I'll provide the query
    Write the querys in the following format: 1. query1\n2. query2\n3. query3 ...
    If you don't think the query needs to be divided, you don't have to.

    Example:
    Query: Who was the Super Bowl MVP in 1979 and 1980?
    You:
    1. Who was the Super Bowl MVP in 1979?
    2. Who was the Super Bowl MVP in 1980?

    Example:
    Query: Are Chris Marker and Yvonne Rainer American?
    You:
    1. Are Chris Marker American?
    2. Are Yvonne Rainer American?
"""

In [7]:
messages = [[
    {"role": "system","content": f"{system_message}"},
    {"role": "user", "content": e}] for e in sampler]

In [8]:
def process_data_gpt4(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=prompt,
        temperature=0.5,
        max_tokens=128,
        top_p=1,
        stop=None,
    )
    return response['choices'][0]['message']['content']

def send_messages_gpt4(messages):
    batch_size = 10
    answers = []

    for i in tqdm(range(0, len(messages), batch_size)):
        batch = messages[i:i + batch_size]

        for message in batch:
            output = process_data_gpt4(message)
            answers.append(output)

        if i + batch_size < len(messages):
            time.sleep(10)

    return answers

In [9]:
answers = send_messages_gpt4(messages)
combined_dataset = [{'query': user, 'decomposed_query': output.split('\n')} for user, output in zip(sampler, answers)]

100%|██████████| 31/31 [11:01<00:00, 21.34s/it]


In [10]:
combined_dataset[0]
save_qa_path = "/content/drive/MyDrive/NER/Multihop_gpt_decompose.json"

with open(save_qa_path, 'w') as file:
    json.dump(combined_dataset, file, indent=4, ensure_ascii=False)
